In [5]:
import pandas as pd
from evaluate import load

In [4]:
df_audio = pd.read_parquet('./data/parquets/Whisper_SNR_WER.gzip')

In [9]:
print(df_audio.columns.to_list())

['audioname', 'dataset', 'ref_orig', 'sampling_rate', 'audiopath_bigos', 'audiopath_local', 'noise_path', 'noise_class', 'normalised_audio_path', 'normalised_noise_path', 'audio_SNR_100_path', 'audio_SNR_50_path', 'audio_SNR_25_path', 'audio_SNR_10_path', 'audio_SNR_5_path', 'audio_SNR_0.1_path', 'audio_SNR_-1_path', 'audio_SNR_-3_path', 'audio_SNR_-10_path', 'WER_audio_SNR_100_path', 'WER_audio_SNR_50_path', 'WER_audio_SNR_25_path', 'WER_audio_SNR_10_path', 'WER_audio_SNR_5_path', 'WER_audio_SNR_0.1_path', 'WER_audio_SNR_-1_path', 'WER_audio_SNR_-3_path', 'WER_audio_SNR_-10_path']


### Whisper SNR evaluation

In [14]:
snr_list = ['WER_audio_SNR_100_path', 'WER_audio_SNR_50_path', 'WER_audio_SNR_25_path', 'WER_audio_SNR_10_path', 'WER_audio_SNR_5_path', 'WER_audio_SNR_0.1_path', 'WER_audio_SNR_-1_path', 'WER_audio_SNR_-3_path', 'WER_audio_SNR_-10_path']
for snr_column in snr_list:
    wer = load("wer")
    snr_value_string = '_'.join(snr_column.split('_')[1:3])
    wer_score = wer.compute(predictions=df_audio[snr_column], references=df_audio['ref_orig'])
    print( f"Whisper model WER score for {snr_value_string}: {round(wer_score * 100,4)}")

Whisper model WER score for audio_SNR: 19.3399
Whisper model WER score for audio_SNR: 19.584
Whisper model WER score for audio_SNR: 22.4602
Whisper model WER score for audio_SNR: 28.2408
Whisper model WER score for audio_SNR: 32.7553
Whisper model WER score for audio_SNR: 42.6143
Whisper model WER score for audio_SNR: 46.2866
Whisper model WER score for audio_SNR: 54.5155
Whisper model WER score for audio_SNR: 98.1438


In [7]:
df_audio

,audioname,dataset,ref_orig,sampling_rate,audiopath_bigos,audiopath_local,noise_path,noise_class,normalised_audio_path,normalised_noise_path,...,audio_SNR_-10_path,WER_audio_SNR_100_path,WER_audio_SNR_50_path,WER_audio_SNR_25_path,WER_audio_SNR_10_path,WER_audio_SNR_5_path,WER_audio_SNR_0.1_path,WER_audio_SNR_-1_path,WER_audio_SNR_-3_path,WER_audio_SNR_-10_path
0,fair-mls-20-train-0009-04739,fair-mls-20,tam nocne włóczęgi wołano z dachów jeżeli nie ...,16000,fair-mls-20-train-0009-04739.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold9\79089-0-0-106.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,hej wy tam nocnemarki wołano z dachów jeżeli ...,hej wy tam nocnemarki wołano z dachów jeżeli ...,"Hej, wy tam, nocne marki! wołano z dachów. Je...","— Hej, wy tam, nocne marki! — wołano z dachów...","Hej, wy tam, nocne marki! Wołamy z dachu. Jeż...","Hej, byj tam, nocne marki! Chyła mu z dachu. ...","Hej, witam, Nocne Marki! Chyba nie zdarzył. J...","Hej, witam, Marcin. Nie mam uwagi. Jeżeli nie...",Dzięki za oglądanie!
1,pjatk-clarin_studio-15-train-0488-00003,pjatk-clarin_studio-15,w pracy studenci chcieliby przede wszystk...,16000,pjatk-clarin_studio-15-train-0488-00003.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold10\167750-4-1-0.wav,drilling,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\pjatk-clarin_s...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,W pracy studenci chcieliby przede wszystkim o...,Pracy studenci chcieliby przede wszystkim ods...,Pracy studenci chcieliby przede wszystkim odb...,Pracy studentów chcieliby przede wszystkim od...,Przede wszystkim chcieliby studenci realizowa...,Dziękuję.
2,fair-mls-20-train-0009-05501,fair-mls-20,co to znaczy sam siebie zapytywał faraon czy g...,16000,fair-mls-20-train-0009-05501.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold3\165039-7-5-0.wav,jackhammer,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,co to znaczy sam siebie zapytywał faraon czy ...,co to znaczy sam siebie zapytywał faraon czy ...,co to znaczy sam siebie zapytywał faraon czy ...,Co to znaczy? sam siebie zapytywał Faraon. Cz...,Co to znaczy? sam siebie zapytał Faraon. Czy ...,Co to znaczy? Sam siebie zapytał paraon. Czy ...,To co znasz? Tam siebie zapytował paraon. Prz...,"Otóż, pan się wychowywał parami przy koło lud...",Dziękuje za oglądanie.
3,fair-mls-20-train-0021-01519,fair-mls-20,tylko na piaszczystem wybrzeżu lub na łąkach b...,16000,fair-mls-20-train-0021-01519.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold1\57320-0-0-22.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./data/mixed_recordings/normalised_recordings/...,...,./data/mixed_recordings/SNR_-10\fair-mls-20-tr...,tylko na piaszczystem wybrzeżu lub na łąkach ...,tylko na piaszczystem wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...,Tylko na piaszczystym wybrzeżu lub na łąkach ...
4,pjatk-clarin_studio-15-train-0335-00001,pjatk-clarin_studio-15,zaokrągla uziemienie księdzu liźnięcie rol...,16000,pjatk-clarin_studio-15-train-0335-00001.wav,C:\Users\Eryk\.cache\huggingface\datasets\down...,.\data\UrbanSound8K\audio\fold4\146709-0-0-20.wav,air_conditioner,./data/mixed_recordings/normalised_recordings/...,./